In [1]:
import os, sys, numpy as np, pandas as pd

# Neural Net imports
from sklearn.preprocessing import  LabelEncoder
from tensorflow.keras.models import load_model

# Import library with current code functions
sys.path.append(os.path.join("..", "lib"))
import neural_network_functions as neural_net_fun, general_functions as gf, files_paths as fp

2024-05-27 14:19:51.405513: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 14:19:51.408897: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 14:19:51.451903: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 14:19:52.354274: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Get the VD_LABELED files paths for your inference videos

In [2]:
FILE_LIST_VD_INFO = gf.find_files_in_all_subdirectories([fp.DATASET_YT, fp.DATASET_LOCAL], fp.VD_INFO)
FILE_LIST_VD_LABELED = gf.find_files_in_all_subdirectories([fp.DATASET_YT, fp.DATASET_LOCAL], fp.VD_LABELED_L0)

## Concatenate all inference CSV with labeled series

In [3]:
# Initialize an empty DataFrame
inference_data = pd.DataFrame()

for current_csv_path in FILE_LIST_VD_LABELED:
    current_data = pd.read_csv(current_csv_path)
    inference_data = pd.concat([inference_data, current_data], ignore_index=True)

## Load your model and preprocess your data

In [4]:
# Path to the trained model (choose the noise applied model or the default one)
model_path = 'lstm_time_series_model.keras'

# Preprocess your data
cleaned_inference_data = neural_net_fun.preprocess_data(inference_data)

## Make the inference

In [5]:
# Generate inferences
predicted_classes = neural_net_fun.generate_inferences(cleaned_inference_data, model_path)

# Decode the predicted classes
predicted_labels = neural_net_fun.decode_predictions(predicted_classes, cleaned_inference_data)

# Print the predicted labels
print(predicted_labels)

2024-05-27 14:19:53.975534: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


2371/2371 ━━━━━━━━━━━━━━━━━━━━ 2s 846us/step
['no_emotion' 'no_emotion' 'no_emotion' ... 'no_emotion' 'no_emotion'
 'no_emotion']


## Evaluate the inferences

In [6]:
# Assuming you have true labels for the new data for testing purposes
true_labels = cleaned_inference_data['target'].values  # Replace this with your true labels if available

# Decode the true labels to match the format of predicted labels
label_encoder = LabelEncoder()
true_labels_encoded = label_encoder.fit_transform(true_labels)

# Evaluate the predictions
neural_net_fun.evaluate_inferences(true_labels_encoded, predicted_classes)

Accuracy: 0.95

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        28
           1       0.95      1.00      0.98     72281
           2       0.00      0.00      0.00      3558

    accuracy                           0.95     75867
   macro avg       0.32      0.33      0.33     75867
weighted avg       0.91      0.95      0.93     75867


Confusion Matrix:
[[    0    28     0]
 [    0 72281     0]
 [    0  3558     0]]
